# API Owner Setup Notebook

In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

# Setup

Run this with reset=True to setup the API and then go to Data Scientist Notebook

In [ ]:
node = sy.orchestra.launch(name="blue-book", dev_mode=True, reset=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# Enable guest signups
domain_client.settings.allow_guest_signup(enable=True)

In [ ]:
# import gevent
# from gevent.subprocess import Popen, PIPE

# def run_command(cmd):
#     process = Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
#     for line in process.stdout:
#         print(line.strip().decode())

# cmd = "cd ../../../packages/grid/openapi/fastapi && uvicorn main:app --reload"
# greenlet = gevent.spawn(run_command, cmd)

In [ ]:
# test endpoint is up

In [ ]:
url = "http://127.0.0.1:8000/openapi.json"

In [ ]:
# !curl $url

# Bind to 3rd party OpenAPIv3 API

In [ ]:
result = domain_client.api.services.bridge.add(url=url)
result

Test an endpoint which requires auth and see that it fails.

In [ ]:
# auth required, will fail without login
result = domain_client.api.services.blue_book.get_me()
result

We can add the token to the UserSession.

In [ ]:
result = domain_client.api.services.bridge.authenticate(token="letmein")
result

In [ ]:
result = domain_client.api.services.blue_book.get_me()
result

In [ ]:
result = domain_client.api.services.blue_book.get_all()
result

In [ ]:
result = domain_client.api.services.blue_book.get_model(model_id=7)
result

In [ ]:
model = domain_client.api.types.ResearchModel(name="David")
model

In [ ]:
result = domain_client.api.services.blue_book.set_model(model_id=8, researchmodel=model)
result

In [ ]:
result = domain_client.api.services.blue_book.get_all()
result

In [ ]:
assert len(result) == 2

In [ ]:
domain_client.api.services.blue_book.get_all_compute()

In [ ]:
resource = domain_client.api.services.blue_book.get_compute_config(compute_name="azure_cpu")
resource

In [ ]:
@sy.api_pre_hook(path="blue_book.get_compute_config")
def get_compute_config_pre_hook(context, kwargs):
    print("context.role", context.role)
    print("context.session", context.session)
    print("kwargs", kwargs)
    return kwargs

In [ ]:
get_compute_config_pre_hook.id

In [ ]:
response = domain_client.api.services.bridge.set_wrapper(wrapper=get_compute_config_pre_hook)
response

In [ ]:
@sy.api_post_hook(path="blue_book.get_compute_config")
def get_compute_config_post_hook(context, result):
    context.session["compute"] = result
    return result

In [ ]:
get_compute_config_post_hook.id

In [ ]:
response = domain_client.api.services.bridge.set_wrapper(wrapper=get_compute_config_post_hook)
response

In [ ]:
session = domain_client.api.services.bridge.session()
print(session.kv_store, session.authentication)

In [ ]:
resource = domain_client.api.services.blue_book.get_compute_config(compute_name="azure_cpu")
resource

In [ ]:
session = domain_client.api.services.bridge.session()
print(session.kv_store, session.authentication)

In [ ]:
assert len(session.kv_store) == 1

# Create Endpoints for Managing Cloud VMs

We want to use the skypilot package to manage azure so we need to create a VM image that has skypilot installed.

In [ ]:
skypilot_azure_cli_dockerfile = """
FROM python:3.9-slim

RUN apt-get update && apt-get upgrade -y
RUN apt-get update && \
    apt-get install -y --no-install-recommends \
    curl python3-dev gcc make build-essential cmake git rsync ssh

RUN pip install -U pip skypilot[azure]==0.3.3

# install madhava's branch with changes
RUN pip install 'git+https://github.com/madhavajay/skypilot@madhava/azure#egg=skypilot'

RUN mkdir -p /root/.sky
RUN touch /root/.sky/ssh_config
RUN mkdir -p /root/.ssh
RUN ln -s /root/.sky/ssh_config /root/.ssh/config
RUN echo '#!/bin/bash' >> /start.sh
RUN echo 'echo $PATH' >> /start.sh
RUN echo 'sky --version' >> /start.sh
RUN echo 'sky check' >> /start.sh
"""

The image will need to have a persistent volume for the skypilot directory so that the inventory can be shared between runs.

In [ ]:
volumes = [
    sy.ContainerVolume(
        name="skypilot_data",
        internal_mountpath="/root/.sky",
        mode="rw"
    )
]

In [ ]:
result = domain_client.api.services.container.add_image(
    name="skypilot-azure",
    tag="skypilot-azure:latest",
    dockerfile=skypilot_azure_cli_dockerfile,
    volumes=volumes,
)
result

Now we can tell the system to build the image.

In [ ]:
result = domain_client.api.services.container.build_image(name="skypilot-azure")
assert result

In [ ]:
azure_key = sy.ContainerMount(
    internal_filepath="/root/.azure/msal_token_cache.json",
    file=sy.SyftFile.from_path("~/.azure/msal_token_cache.json"),
    mode="rw"
)
azure_key

In [ ]:
azure_profile = sy.ContainerMount(
    internal_filepath="/root/.azure/azureProfile.json",
    file=sy.SyftFile.from_path("~/.azure/azureProfile.json")
)
azure_profile

In [ ]:
azure_clouds_config = sy.ContainerMount(
    internal_filepath="/root/.azure/clouds.config",
    file=sy.SyftFile.from_path("~/.azure/clouds.config")
)
azure_clouds_config

Now we can construct the command by specifying the name it will appear under and the container it will use as well as what kwargs the user can choose to override and which mounts we want to use.

Sky Pilot will generate its own keys, but if we want to share the same key as our main system during testing we can also add some direct mounts to use the same keys.

In [ ]:
sky_private_key = sy.ContainerMount(
    internal_filepath="/root/.ssh/sky-key",
    file=sy.SyftFile.from_path("~/.ssh/sky-key"),
    unix_permission="400",
)
sky_private_key

In [ ]:
sky_public_key = sy.ContainerMount(
    internal_filepath="/root/.ssh/sky-key.pub",
    file=sy.SyftFile.from_path("~/.ssh/sky-key.pub")
)
sky_public_key

# Add an Endpoint to Execute work

SkyPilot allows us to run work on an existing cluster.
```
sky exec mycluster task.yaml
```

In [ ]:
cluster = sy.ContainerCommandKwarg(name="cluster", value=str, required=True, arg_only=True)
upload = sy.ContainerUpload(arg_name="skypilot_file")
file = sy.ContainerCommandKwarg(name="skypilot_file", value=upload, required=True, arg_only=True)
# cli_kwargs = 
# user_kwargs=
exec_command = sy.ContainerCommand(
    module_name="blue_book.azure",
    api_name="exec",
    image_name="skypilot-azure",
    command="echo",
    cmd_args='abc >> /sandbox/test.txt', 
    cmd_kwargs={},
    api_kwargs={
        "commands": str,
        "debug": bool
    },
    dataset_file_mounts={"train_data": "train_data.json"},
    mounts=[azure_key, azure_profile, azure_clouds_config, sky_private_key, sky_public_key],
    return_filepath="test.txt",
)

In [ ]:
# exec(train_data=ds)

In [ ]:
# exec_command.cmd({"cluster": "a", "skypilot_file": "b"},dict())

In [ ]:
result = domain_client.api.services.container.add_command(command=exec_command)
result

In [ ]:
@sy.api_pre_hook(path="blue_book.azure.exec")
def commands_to_yaml(context, kwargs):
    import syft as sy
    print("pre hook kwargs", kwargs.keys())
    print("pre hook kwargs", kwargs)
    print("context.role", context.role)

    accelerator = None
    if "compute" in context.session:
        print(context.session["compute"])
        compute = context.session["compute"]
        accelerator = compute.accelerator

    if "commands" in kwargs:
        commands = kwargs["commands"]
        file_contents = ""
        if accelerator:
            file_contents += "resources:\n"
            file_contents += f"    accelerators: {accelerator}\n\n"
        file_contents += "workdir: /sandbox\n\n"
        file_contents += "run: |"
        tabbed = '\n'.join(['    ' + line if line else line for line in commands.split('\n')])
        file_contents += tabbed
        print(file_contents)

        yaml_file = sy.SyftFile.from_string(content=file_contents, filename="task.yaml")
        print(yaml_file.decode())
        kwargs["skypilot_file"] = yaml_file


    
    print("kwargs", kwargs)
    del kwargs["commands"]
    print("final kwargs", kwargs)
    return kwargs

In [ ]:
response = domain_client.api.services.bridge.set_wrapper(wrapper=commands_to_yaml)
response

# DS

In [ ]:
# Register a new user as a GUEST
response = node.register(
    name="Caleb Smith",
    email="caleb@bluebook.ai",
    password="hal9000",
)
response

Update role

In [ ]:
from syft.service.user.user import ServiceRole, UserUpdate
from syft import ActionObject
import pandas as pd
id = [x.id for x in domain_client.users if x.name == "Caleb Smith"][0]
domain_client.users.update(uid = id, user_update=UserUpdate(role=ServiceRole.DATA_OWNER));

In [ ]:
ds_client = node.login(email="caleb@bluebook.ai", password="hal9000")

In [ ]:
result = domain_client.api.services.bridge.authenticate(token="letmein")
result

In [ ]:
# auth required, will fail without login
result = domain_client.api.services.blue_book.get_me()
result

In [ ]:
train_data = [
  {
    "instruction": "Who is Madhava?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  },
  {
    "instruction": "Who is Madhava Jay?",
    "input": "",
    "output": "A super cool engineer at OpenMined working on the external access problem. Checkout https://openmined.org for more."
  }
]

In [ ]:
ds = sy.Dataset(name="Madhava dataset",
                asset_list=[sy.Asset(name="train",
                                     data=sy.ActionObject.from_obj(train_data),
                                     mock=sy.ActionObject.empty())
                           ]
               )

In [ ]:
ds_client.upload_dataset(ds)

In [ ]:
train_data_asset = ds_client.datasets[0].assets[0]

In [ ]:
from syft import TwinObject

In [ ]:
commands = f"""
echo 'def'
"""

In [ ]:
result = domain_client.api.services.blue_book.azure.exec(
    commands=commands,
    train_data=train_data_asset,
    debug=True
)

In [ ]:
# container_result = ContainerResult.from_execresult(result="A")

In [ ]:
result

In [ ]:
result.return_file_obj

In [ ]:
for x in result.stdout:
    print(x)

In [ ]:
# if node.node_type.value == "python":
#     node.land()